In [1]:
import os; os.chdir('..')

In [2]:
from tqdm.autonotebook import tqdm

/home/oserbetci/miniconda/envs/icu/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """Entry point for launching an IPython kernel.


In [3]:
from dataloader import data
from dataloader.data import get_tables, MIMICDataset, BinnedTabularFeature, TabularFeature
from dataloader.labels import get_labels
from dataloader.extractors import extract_merged_sentences, extract_sentences, extract_patient_sentences
from dataloader.utils import BinnedEvent, LabelEvent

In [5]:
DEVICE = 'cpu'

dem, chart, lab, output, prescr, inpute = get_tables(['dem',
                                                      'CHARTEVENTS',
                                                      'LABEVENTS', 
                                                      'OUTPUTEVENTS', 
                                                      'PRESCRIPTIONS', 
                                                      'INPUTEVENTS_*'])

labels = get_labels(DEVICE)
tables = [chart, lab, output, prescr, inpute, dem]
train_set = MIMICDataset(datalist_file='train/listfile.csv', datasplit='train', mode='EVAL',
                         tables=tables, labels=labels, use_cache=False)
test_set = MIMICDataset(datalist_file='test/listfile.csv', datasplit='test', mode='EVAL',
                        tables=tables, labels=labels, use_cache=False)

#### generate PATIENT SENTENCES

In [ ]:
def _timestep_transform(self, df, *args, **kwargs):
    breakpoint()
    df['timestep'] = df['Hours']
    return df

# Monkey patch
BinnedTabularFeature._timestep_transform = _timestep_transform

In [ ]:
BIN_STRATEGY = 'kmeans'
DATA_SUFFIX = '.mimic3.train'
N_BINS = 10

_, chart, lab, output, prescr, inpute = get_tables(['CHARTEVENTS', 'LABEVENTS', 'OUTPUTEVENTS', 'PRESCRIPTIONS', 'INPUTEVENTS_*', 'dem'],
                                                   load=True,
                                                   include_source=False,
                                                   n_bins=N_BINS,
                                                   suffix=DATA_SUFFIX,
                                                   strategy=BIN_STRATEGY)

train_set.use_cache = False

tables = [chart, lab, output, prescr, inpute]
extract_patient_sentences(train_set, tables, randomize=False, suffix=f'{DATA_SUFFIX}.{BIN_STRATEGY}.{N_BINS}bins.cloip.patient.label',
                          include_source=False, event_class=BinnedEvent, sep_bin=False)


In [ ]:
BIN_STRATEGY = 'uniform'
DATA_SUFFIX = '.mimic3.train'
N_BINS = 10

_, chart, lab, output, prescr, inpute = get_tables(['CHARTEVENTS', 'LABEVENTS', 'OUTPUTEVENTS', 'PRESCRIPTIONS', 'INPUTEVENTS_*', 'dem'],
                                                   load=True,
                                                   include_source=False,
                                                   n_bins=N_BINS,
                                                   suffix=DATA_SUFFIX,
                                                   strategy=BIN_STRATEGY)

train_set.use_cache = False

tables = [chart, lab, output, prescr, inpute]
extract_patient_sentences(train_set, tables, randomize=False, suffix=f'{DATA_SUFFIX}.{BIN_STRATEGY}.{N_BINS}bins.cloip.patient',
                          include_source=False, event_class=BinnedEvent, sep_bin=False)

In [ ]:
!less ../data/sentences.mimic3.train.uniform.10bins.cloip.patient.train.txt


#### generate concatenated hourly sentences

In [ ]:
# Undo monkey patch
import importlib
importlib.reload(data)
BinnedTabularFeature._timestep_transform = data.BinnedTabularFeature._timestep_transform

In [ ]:
BIN_STRATEGY = 'uniform'
DATA_SUFFIX = '.mimic3.train'
N_BINS = 10

dem, chart, lab, output, prescr, inpute = get_tables(['CHARTEVENTS', 'LABEVENTS', 'OUTPUTEVENTS', 'PRESCRIPTIONS', 'INPUTEVENTS_*', 'dem'],
                                                     include_source=False,
                                                     n_bins=N_BINS,
                                                     suffix=DATA_SUFFIX,
                                                     strategy=BIN_STRATEGY,
                                                     load=True)

train_set.use_cache = True

tables = [dem, chart, lab, output, prescr, inpute]
for table in tables:
    table.fit(train_set)
    table.save()
    
tables = [chart, lab, output, prescr, inpute]
extract_merged_sentences(train_set, tables, randomize=False, suffix=f'{DATA_SUFFIX}.{BIN_STRATEGY}.{N_BINS}bins.cloip.merged',
                         include_source=False, event_class=BinnedEvent, sep_bin=False)
extract_merged_sentences(train_set, tables, randomize=False, suffix=f'{DATA_SUFFIX}.clo.merged',
                         include_source=False, event_class=BinnedEvent, sep_bin=True)

In [ ]:
BIN_STRATEGY = 'kmeans'
DATA_SUFFIX = '.mimic3.train'
N_BINS = 10

chart, lab, output, prescr, inpute = get_tables(['CHARTEVENTS', 'LABEVENTS', 'OUTPUTEVENTS', 'PRESCRIPTIONS', 'INPUTEVENTS_*'],
                                                include_source=False,
                                                n_bins=N_BINS,
                                                suffix=DATA_SUFFIX,
                                                strategy=BIN_STRATEGY,
                                                load=True)

train_set.use_cache = True

# generate cache
tables = [chart, lab, output, prescr, inpute]
for table in tables:
    table.fit(train_set)
    table.save()
    
tables = [chart, lab, output]
extract_merged_sentences(train_set, tables, randomize=False, suffix=f'{DATA_SUFFIX}.{BIN_STRATEGY}.{N_BINS}bins.clo.merged',
                         include_source=False, event_class=BinnedEvent, sep_bin=False)
extract_merged_sentences(train_set, tables, randomize=True, suffix=f'{DATA_SUFFIX}.{BIN_STRATEGY}.{N_BINS}bins.cloip.random_merged',
                         include_source=False, event_class=BinnedEvent, sep_bin=False)

In [ ]:
print('DONE')

# Create Test Vocab

In [ ]:
DATA_SUFFIX = '.mimic3.test'
BIN_STRATEGY = 'kmeans'
N_BINS = 10

def _timestep_transform(self, df, *args, **kwargs):
    df['timestep'] = df['Hours']
    return df

# Monkey patch
BinnedTabularFeature._timestep_transform = _timestep_transform

_, chart, lab, output, prescr, inpute = get_tables(['CHARTEVENTS', 'LABEVENTS', 'OUTPUTEVENTS', 'PRESCRIPTIONS', 'INPUTEVENTS_*', 'dem'],
                                                   load=True,
                                                   include_source=False,
                                                   n_bins=N_BINS,
                                                   suffix=DATA_SUFFIX,
                                                   strategy=BIN_STRATEGY)

tables = [chart, lab, output, prescr, inpute]
extract_patient_sentences(test_set, tables, randomize=False, suffix=f'{DATA_SUFFIX}.{BIN_STRATEGY}.{N_BINS}.cloip.patient',
                          include_source=False, event_class=BinnedEvent, sep_bin=False)


Automatic pdb calling has been turned ON
